In [ ]:
%pip install --upgrade llama-index        # core
%pip install --upgrade llama-index-llms-google-genai  # Google / Gemini LLM integration
%pip install --upgrade llama-index-embeddings-google-genai  # embeddings via Google GenAI
%pip install --upgrade google-generativeai  # underlying Google SDK
%pip install --upgrade pinecone-client      # if using Pinecone
    

In [ ]:
import os
from pinecone import Pinecone
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,Settings, get_response_synthesizer
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

In [ ]:
# Force Google models globally BEFORE anything else (prevents OpenAI
os.environ.pop("OPENAI_API_KEY", None) # ensure no accidental

Settings.embed_model = GoogleGenAIEmbedding(model_name="models/text-embedding-004",api_key="")
Settings.llm = GoogleGenAI(
    model="gemini-2.5-flash",
    api_key="",
    use_async=True  
)


In [ ]:
# =============================================
# Data layer existing index with 004 emebeding model
# =============================================

INDEX_NAME = "coffeeindex"
pc = Pinecone(api_key="")
pc_index = pc.Index(INDEX_NAME)
vstore = PineconeVectorStore(pinecone_index=pc_index)

In [ ]:
os.getcwd()

In [ ]:
# Local PDFs (NO Pinecone upsert; kept in-memory)
docs = SimpleDirectoryReader("./GenAI/GenAI-NoteBooks/coffee_pages").load_data()

In [ ]:
# Build indexes (use Settings.embed_model)
vector_idx = VectorStoreIndex.from_vector_store(vstore) #Pinecone-backed
local_idx = VectorStoreIndex.from_documents(docs) #in-memory for PDF

In [ ]:
# ============================ Query layer
#============================ (Optional for now)
# Auto-fuse Pinecone + PDFs; no query expansion (no hidden LLM calls in retrieval)
pinecone_ret = vector_idx.as_retriever(similarity_top_k=5)
pdf_ret = local_idx.as_retriever(similarity_top_k=5)
hybrid = QueryFusionRetriever(
 retrievers=[pinecone_ret, pdf_ret],
 similarity_top_k=5,
 mode="reciprocal_rerank",
 num_queries=1,
 use_async=False,
)

In [ ]:
qe = RetrieverQueryEngine.from_args(
 retriever=hybrid,
 response_synthesizer=get_response_synthesizer(llm=Settings.llm,
response_mode="compact")
)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
print(qe.query("Using both PDFs and the vector KB, explain desi brewing in 3 bullets."))